In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:
img=cv2.imread('../input/intel-image-classification/seg_train/seg_train/buildings/1001.jpg')
plt.imshow(img)
#print(img)

In [ ]:
img_path=[]
categoryy=[]
path='../input/intel-image-classification/seg_train/seg_train/'
categories=['buildings','forest','glacier','mountain','sea','street']
for category in categories:
    abs_path=os.path.join(path,category)
    for _,_,filenames in os.walk(abs_path):
        for file in filenames:
            img_path.append(os.path.join(abs_path,file))
            categoryy.append(category)

In [ ]:
img_path_test=[]
categoryy_test=[]
path='../input/intel-image-classification/seg_test/seg_test/'
categories=['buildings','forest','glacier','mountain','sea','street']
for category in categories:
    abs_path=os.path.join(path,category)
    for _,_,filenames in os.walk(abs_path):
        for file in filenames:
            img_path_test.append(os.path.join(abs_path,file))
            categoryy_test.append(category)


In [ ]:
print(img_path[4950:5000])

In [ ]:
print(categoryy[4950:5000])

In [ ]:
df=pd.DataFrame()
df['img_path']=img_path
df['img_label']=categoryy

In [ ]:
df_test=pd.DataFrame()
df_test['img_path']=img_path_test
df_test['img_label']=categoryy_test

In [ ]:
df_test.head()

In [ ]:
plt.imshow(cv2.imread(df['img_path'][1]))

In [ ]:
df['img_label'][1]

In [ ]:
from sklearn.utils import shuffle

In [ ]:
df=shuffle(df)
df.reset_index()

In [ ]:
from sklearn.model_selection import train_test_split
df_train,validation=train_test_split(df,train_size=0.9,random_state=42)

In [ ]:
df_test=shuffle(df_test)

In [ ]:
df_train.head()

In [ ]:
df_train=df_train.reset_index()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size=100
img=cv2.resize(cv2.imread(df_train['img_path'][1]),(img_size,img_size))
plt.imshow(img)

In [ ]:
train_datagen=ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
val_datagen=ImageDataGenerator(rescale = 1.0/255.)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)


In [ ]:
train_gen=train_datagen.flow_from_dataframe(dataframe=df_train,x_col='img_path',y_col='img_label',target_size=(100,100),batch_size=128,class_mode='categorical',shuffle=True)
val_gen=test_datagen.flow_from_dataframe(dataframe=validation,x_col='img_path',y_col='img_label',target_size=(100,100),batch_size=128,class_mode='categorical',shuffle=True)
test_gen=test_datagen.flow_from_dataframe(dataframe=df_test,x_col='img_path',y_col='img_label',target_size=(100,100),batch_size=128,class_mode='categorical',shuffle=True)

In [ ]:
val_gen.classes[:10]

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="relu",input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(Conv2D(128,(3,3),padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(Conv2D(256,(3,3),padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(32,(3,3),padding="same",activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

# model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(6))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
history = model.fit(train_gen, validation_data = val_gen, epochs = 35, verbose = 1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

In [ ]:
model.evaluate(test_gen)

In [ ]:
model.summary()

In [ ]:
model.save("model.h5")

In [ ]:
path1='../input/intel-image-classification/seg_pred/seg_pred/'
tester=[]
for _,_,files in os.walk(path1):
    for file in files:
        tester.append(os.path.join(path1,file))
        

In [ ]:
tester[:10]

In [ ]:
df_tester=pd.DataFrame()
df_tester['img_path']=tester

In [ ]:
def load_images(images_folder, img_size = (100,100), scale=True, pred_set=False):

    # Store paths to images
    image_path = []
    for dirname, _, filenames in os.walk(images_folder):
        for filename in filenames:
            image_path.append(os.path.join(dirname, filename))

    print("There are {} images in {}".format(len(image_path), images_folder))
    
    # Load images and associated labels
    images = []
    labels = []

    for path in image_path:

        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (100,100)) # Resize the images

        img = np.array(img)
        images.append(img)
        if not pred_set: # pred doesn't have label
            labels.append(categories[path.split('/')[-2]]) # last folder before the image name is the category

    images = np.array(images)  
    images = images.astype(np.int64)
    
    if scale:
        images = images/255 # scale
        
    return image_path, images, np.asarray(labels)

In [ ]:
def predict_class(img):
    # Resize
    img = img.reshape(1,100,100,3)
    # Predict
    predictions = model.predict(img)
    true_prediction = [tf.argmax(pred) for pred in predictions]
    true_prediction = np.array(true_prediction)
    
    # Return label corresponding to predicted index
    return true_prediction

In [ ]:
images_pred_folder = '../input/intel-image-classification/seg_pred/seg_pred'
_, X_pred, y_pred = load_images(images_pred_folder, img_size=img_size, scale=True, pred_set=True)

X_pred = np.array(X_pred).reshape(-1,100,100,3)
X_pred.shape

In [ ]:
plt.imshow(X_pred[69])

In [ ]:
predict_class(X_pred[69])

In [ ]:
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('model.h5')

In [ ]:
tf.keras.backend.clear_session()